#INITIALIZING DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#open the specific path


Mounted at /content/gdrive


#LOAD DATASET

In [ ]:
import numpy as np
import scipy.io

In [ ]:
DATASET='AWA2'
PATH='/content/gdrive/MyDrive/tfvaegan/tfavegan/datasets' #change with your path
ATTRIBUTES_PATH = PATH + '/' + DATASET + '/att_splits.mat'


att_splits = scipy.io.loadmat(ATTRIBUTES_PATH)
attrs = att_splits['att'].transpose()

CLASS=50
ATTRIBUTES=85
TEST=10
#parameters of AWA dataset, change for other datasets

#More zeros unseen class split

In [ ]:
ax=np.zeros(CLASS)


for i,arr in enumerate(attrs):
   ax[i]=np.count_nonzero(arr == 0)


zeros_split=[]
zeros_split.append((-ax).argsort()[:72])
print("More zeros unseen class split: ",zeros_split)

More zeros unseen class split:  [array([ 8, 36, 17,  4, 37, 13, 22,  2, 39, 27, 31, 45, 43, 18,  6, 44, 11,
       30, 16, 35, 28, 49,  0,  1, 25,  3, 40, 42, 21, 10, 47, 46, 15, 23,
       32, 14, 33,  7, 41,  5, 20, 26, 12,  9, 38, 29, 48, 19, 24, 34])]


#More discriminative unseen class split

In [ ]:
rank=[]
sorted_att=[]
attrsT=np.transpose(attrs)



for a in attrsT:
  rank.append((a).argsort()[:CLASS].tolist())
  sorted_att.append(np.sort(a))


discr=np.zeros(CLASS)
f=np.zeros(CLASS)
for i in range(CLASS):
    f[i]=pow((i-(CLASS/2)),2)
    

for i in range(len(rank)):
    for j in range(len(rank[0])):      
        discr[rank[i][j]]+=f[j]*abs(sorted_att[i][j])
       
sorted_d=np.sort(discr)

print("More discriminative unseen class split: ",np.where(np.isin(discr,sorted_d[CLASS-TEST:CLASS])==True))


More discriminative unseen class split:  (array([ 2,  3,  8, 17, 22, 23, 26, 27, 38, 46]),)


#The furthest unseen class

###Build the nearness matrix

In [ ]:
def subtract(colA,colB):
    distance=0
    for i in range(len(colA)):
        distance+=np.abs(colA[i]-colB[i])
        
    return distance
nearness=np.zeros((CLASS,CLASS))
for i in range(CLASS):  
    for j in range(CLASS):        
        if(j==i):
          k=j+1
        else:
          k=j
        if(k!=CLASS):
          if(i<k):
            nearness[i][k]=subtract(attrs[i],attrs[k])
          else:
            nearness[i][k]=nearness[k][i]


###Find the split

In [ ]:
def get_where(c,p):
    arr=np.isin(c,p)
    result=np.where(arr==False)
    return result[0]


def max_guad(pos,unseen):
    min=counter[unseen[0]]-2*nearness[unseen[0]][pos]-nearness[unseen[0]][unseen].sum()
    temp=unseen[0]
    for x in unseen:     
      a=(counter[x]-2*nearness[x][pos]-nearness[x][unseen].sum())
      if(a<min):
        min=a
        temp=x
    if(min<10000000000000000):
        swap[pos]=temp

    
    return min
def fill_gain(seen, unseen, gain):
    for i,v in enumerate(nearness[seen]):
      m_guad=max_guad(seen[i],unseen)      
      gain[seen[i]]=v[seen].sum()-nearness[seen[i]][unseen].sum()-m_guad
    return gain
def fill_counter(unseen,counter):
    for i,v in enumerate(nearness[unseen]):
      counter[unseen[i]]=v.sum()-v[unseen].sum()
    return counter
swap=np.zeros(CLASS)
z=[]

for i in range(TEST):
  z.append(i)
unseen=np.array(z)

all_class=range(CLASS)
seen=get_where(all_class,unseen)
counter=np.zeros(CLASS)
gain=np.zeros(CLASS)
counter=fill_counter(unseen,counter)

gain=fill_gain(seen,unseen,gain)

while (gain[np.argmax(gain)]>0):
  
  max_gain=np.argmax(gain)
  index=np.where(seen==max_gain)[0] 
  seen=np.insert(np.delete(seen,index),index,swap[max_gain])
  index=np.where(unseen==swap[max_gain])[0] 
  unseen=np.insert(np.delete(unseen,index),index,max_gain)
  
  counter=np.zeros(CLASS)
  gain=np.zeros(CLASS)
  
  counter=fill_counter(unseen,counter)
  gain=fill_gain(seen,unseen,gain)
  




print('The furthest unseen class split: ',unseen)



  







FINALE:
unseen:  [23 27  2 18 13 35 46 17  8 49]


In [ ]:
def sottrai(colA,colB):
    somma=0
    for i in range(len(colA)):
        g=np.abs(colA[i]-colB[i])
        somma+=g
    return somma

        
attrsT=np.transpose(attrs)
vicinanza=np.zeros((ATTRIBUTI,ATTRIBUTI))
for i in range(ATTRIBUTI):  
    for j in range(ATTRIBUTI):        
        if(j==i):
          k=j+1
        else:
          k=j
        if(k!=ATTRIBUTI):
          vicinanza[k][i]=sottrai(attrsT[i],attrsT[k])

arr=[]
for i,s in enumerate(vicinanza):
    temp=s
    temp[i]=1000
    for j in range(ATTRIBUTI):
        arr.append((i,j,temp[j]))

arr1=[]


arr.sort(key = lambda x: x[2] )
for t in range(100):
    arr1.append(arr[2*t])
for ar in arr1:

    print(ar)
def stats(arr):
    a=np.zeros(ATTRIBUTI)
    for s in arr:
      (i,j,k)=s
      a[i]+=1
      a[j]+=1
    print((-a).argsort()[:10])
    print(a)

stats(arr1)






